# Dynamic-DeepHit Tutorial

### by Changhee Lee

In [1]:
_EPSILON = 1e-08

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

import matplotlib.pyplot as plt

import import_data as impt

from class_DeepLongitudinal import Model_Longitudinal_Attention

from utils_eval             import c_index, brier_score
from utils_log              import save_logging, load_logging
from utils_helper           import f_get_minibatch, f_get_boosted_trainset

warnings.filterwarnings('ignore')

C:\Users\excen\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\excen\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\excen\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\excen\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [23]:
def _f_get_pred(sess, model, data, data_mi, pred_horizon):
    '''
        predictions based on the prediction time.
        create new_data and new_mask2 that are available previous or equal to the prediction time (no future measurements are used)
    '''
    new_data    = np.zeros(np.shape(data))
    new_data_mi = np.zeros(np.shape(data_mi))

    meas_time = np.concatenate([np.zeros([np.shape(data)[0], 1]), np.cumsum(data[:, :, 0], axis=1)[:, :-1]], axis=1)

    for i in range(np.shape(data)[0]):
        last_meas = np.sum(meas_time[i, :] <= pred_horizon)

        new_data[i, :last_meas, :]    = data[i, :last_meas, :]
        new_data_mi[i, :last_meas, :] = data_mi[i, :last_meas, :]

    return model.predict(new_data, new_data_mi)


def f_get_risk_predictions(sess, model, data_, data_mi_, pred_time, eval_time):
    
    pred = _f_get_pred(sess, model, data_[[0]], data_mi_[[0]], 0)
    _, num_Event, num_Category = np.shape(pred)
       
    risk_all = {}
    for k in range(num_Event):
        risk_all[k] = np.zeros([np.shape(data_)[0], len(pred_time), len(eval_time)])
            
    for p, p_time in enumerate(pred_time):
        ### PREDICTION
        pred_horizon = int(p_time)
        pred = _f_get_pred(sess, model, data_, data_mi_, pred_horizon)


        for t, t_time in enumerate(eval_time):
            eval_horizon = int(t_time) + pred_horizon #if eval_horizon >= num_Category, output the maximum...

            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
            risk = np.sum(pred[:,:,pred_horizon:(eval_horizon+1)], axis=2) #risk score until eval_time
            risk = risk / (np.sum(np.sum(pred[:,:,pred_horizon:], axis=2), axis=1, keepdims=True) +_EPSILON) #conditioniong on t > t_pred
            
            for k in range(num_Event):
                risk_all[k][:, p, t] = risk[:, k]
                
    return risk_all

### 1. Import Dataset
#####      - Users must prepare dataset in csv format and modify 'import_data.py' following our examplar 'PBC2'

In [54]:
data_mode                   = 'PBC2' 
seed                        = 1234

##### IMPORT DATASET
'''
    num_Category            = max event/censoring time * 1.2
    num_Event               = number of evetns i.e. len(np.unique(label))-1
    max_length              = maximum number of measurements
    x_dim                   = data dimension including delta (1 + num_features)
    x_dim_cont              = dim of continuous features
    x_dim_bin               = dim of binary features
    mask1, mask2, mask3     = used for cause-specific network (FCNet structure)
'''

if data_mode == 'PBC2':
    (x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi), df_raw = impt.import_dataset(norm_mode = 'standard')
    
    # This must be changed depending on the datasets, prediction/evaliation times of interest
    pred_time = [8] # prediction time (in years)
    eval_time = [10] # years evaluation time (for C-index and Brier-Score)
else:
    print ('ERROR:  DATA_MODE NOT FOUND !!!')

_, num_Event, num_Category  = np.shape(mask1)  # dim of mask3: [subj, Num_Event, Num_Category]
max_length                  = np.shape(data)[1]


file_path = '{}'.format(data_mode)

if not os.path.exists(file_path):
    os.makedirs(file_path)

### 2. Set Hyper-Parameters
##### - Play with your own hyper-parameters!

In [55]:
burn_in_mode                = 'ON' #{'ON', 'OFF'}
boost_mode                  = 'ON' #{'ON', 'OFF'}

##### HYPER-PARAMETERS
new_parser = {'mb_size': 128,

             'iteration_burn_in': 3000,
             'iteration': 25000,

             'keep_prob': 0.6,
             'lr_train': 1e-4,

             'h_dim_RNN': 100,
             'h_dim_FC' : 100,
             'num_layers_RNN':3,
             'num_layers_ATT':2,
             'num_layers_CS' :3,

             'RNN_type':'LSTM', #{'LSTM', 'GRU'}

             'FC_active_fn' : tf.nn.relu,
             'RNN_active_fn': tf.nn.tanh,

            'reg_W'         : 1e-5,
            'reg_W_out'     : 0.,

             'alpha' :1.0,
             'beta'  :0.1,
             'gamma' :1.0
}


# INPUT DIMENSIONS
input_dims                  = { 'x_dim'         : x_dim,
                                'x_dim_cont'    : x_dim_cont,
                                'x_dim_bin'     : x_dim_bin,
                                'num_Event'     : num_Event,
                                'num_Category'  : num_Category,
                                'max_length'    : max_length }

# NETWORK HYPER-PARMETERS
network_settings            = { 'h_dim_RNN'         : new_parser['h_dim_RNN'],
                                'h_dim_FC'          : new_parser['h_dim_FC'],
                                'num_layers_RNN'    : new_parser['num_layers_RNN'],
                                'num_layers_ATT'    : new_parser['num_layers_ATT'],
                                'num_layers_CS'     : new_parser['num_layers_CS'],
                                'RNN_type'          : new_parser['RNN_type'],
                                'FC_active_fn'      : new_parser['FC_active_fn'],
                                'RNN_active_fn'     : new_parser['RNN_active_fn'],
                                'initial_W'         : tf.contrib.layers.xavier_initializer(),

                                'reg_W'             : new_parser['reg_W'],
                                'reg_W_out'         : new_parser['reg_W_out']
                                 }


mb_size           = new_parser['mb_size']
iteration         = new_parser['iteration']
iteration_burn_in = new_parser['iteration_burn_in']

keep_prob         = new_parser['keep_prob']
lr_train          = new_parser['lr_train']

alpha             = new_parser['alpha']
beta              = new_parser['beta']
gamma             = new_parser['gamma']

# SAVE HYPERPARAMETERS
log_name = file_path + '/hyperparameters_log.txt'
save_logging(new_parser, log_name)

### 3. Split Dataset into Train/Valid/Test Sets

In [56]:
indecies_mesa = pd.read_csv('R:/PrevMed/projects/MESA_RiskPred/LRPP data/mesa_index_select.csv')
indecies_mesa = np.array(indecies_mesa['study_index'])

df_index = list(range(len(data)))

def train_test_split_index(df, indecies):
    tr_data = df[indecies == False]
    te_data = df[indecies]
    
    return tr_data, te_data

In [57]:
split_option = 'MESA'

if split_option == 'MESA':

    ### Select MESA as Testing set 
    tr_data, te_data = train_test_split_index(data, indecies_mesa)
    tr_data_raw, te_data_raw = train_test_split_index(df_raw, indecies_mesa)
    tr_df_index, te_df_index = train_test_split_index(np.array(df_index), indecies_mesa)
    tr_data_mi, te_data_mi = train_test_split_index(data_mi, indecies_mesa)
    tr_time, te_time = train_test_split_index(time, indecies_mesa)
    tr_label, te_label = train_test_split_index(label, indecies_mesa)
    tr_mask1, te_mask1 = train_test_split_index(mask1, indecies_mesa)
    tr_mask2, te_mask2 = train_test_split_index(mask2, indecies_mesa)
    tr_mask3, te_mask3 = train_test_split_index(mask3, indecies_mesa) 
else:
    ### TRAINING-TESTING SPLIT
    (tr_data, te_data, tr_data_raw, te_data_raw, tr_df_index, te_df_index, tr_data_mi, te_data_mi, tr_time, te_time, tr_label, te_label, 
    tr_mask1,te_mask1, tr_mask2,te_mask2, tr_mask3,te_mask3) = train_test_split(data, df_raw, df_index, data_mi, time, label, mask1, mask2, mask3, test_size=0.3, random_state=seed, stratify = label) 

(tr_data, va_data, tr_data_mi, va_data_mi, tr_time, va_time, tr_label, va_label, 
 tr_mask1,va_mask1, tr_mask2,va_mask2, tr_mask3,va_mask3) = train_test_split(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3, test_size=0.3, random_state=seed, stratify = tr_label) 

if boost_mode == 'ON':
    tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)

In [58]:
##### CREATE DYNAMIC-DEEPHIT NETWORK
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())

INFO:tensorflow:Scale of 0 disables regularizer.


In [59]:
### TRAINING - BURN-IN
if burn_in_mode == 'ON':
    print( "BURN-IN TRAINING ...")
    for itr in range(iteration_burn_in):
        x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
        DATA = (x_mb, k_mb, t_mb)
        MISSING = (x_mi_mb)

        _, loss_curr = model.train_burn_in(DATA, MISSING, keep_prob, lr_train)

        if (itr+1)%1000 == 0:
            print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))


### TRAINING - MAIN
print( "MAIN TRAINING ...")
min_valid = 0.5

for itr in range(iteration):
    x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
    DATA = (x_mb, k_mb, t_mb)
    MASK = (m1_mb, m2_mb, m3_mb)
    MISSING = (x_mi_mb)
    PARAMETERS = (alpha, beta, gamma)

    _, loss_curr = model.train(DATA, MASK, MISSING, PARAMETERS, keep_prob, lr_train)

    if (itr+1)%1000 == 0:
        print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))

    ### VALIDATION  (based on average C-index of our interest)
    if (itr+1)%1000 == 0:        
        risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)
        
        for p, p_time in enumerate(pred_time):
            pred_horizon = int(p_time)
            val_result1 = np.zeros([num_Event, len(eval_time)])
            
            for t, t_time in enumerate(eval_time):                
                eval_horizon = int(t_time) + pred_horizon
                for k in range(num_Event):
                    val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            
            if p == 0:
                val_final1 = val_result1
            else:
                val_final1 = np.append(val_final1, val_result1, axis=0)

        tmp_valid = np.mean(val_final1)

        if tmp_valid >  min_valid:
            min_valid = tmp_valid
            saver.save(sess, file_path + '/model')
            print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))

BURN-IN TRAINING ...
itr: 1000 | loss: 2.0677
itr: 2000 | loss: 1.3464
itr: 3000 | loss: 1.5232
MAIN TRAINING ...
itr: 1000 | loss: 2.5791
updated.... average c-index = 0.8007
itr: 2000 | loss: 2.5021
updated.... average c-index = 0.8046
itr: 3000 | loss: 2.3008
updated.... average c-index = 0.8047
itr: 4000 | loss: 2.1858
updated.... average c-index = 0.8059
itr: 5000 | loss: 2.5463
itr: 6000 | loss: 2.3947
itr: 7000 | loss: 2.7438
updated.... average c-index = 0.8065
itr: 8000 | loss: 1.8497
itr: 9000 | loss: 2.7048
updated.... average c-index = 0.8066
itr: 10000 | loss: 2.1203
itr: 11000 | loss: 1.8567
updated.... average c-index = 0.8089
itr: 12000 | loss: 2.1438
updated.... average c-index = 0.8094
itr: 13000 | loss: 1.6616
itr: 14000 | loss: 1.8909
updated.... average c-index = 0.8112
itr: 15000 | loss: 2.5762
itr: 16000 | loss: 1.6489
itr: 17000 | loss: 2.1490
itr: 18000 | loss: 2.3043
itr: 19000 | loss: 2.6016
updated.... average c-index = 0.8123
itr: 20000 | loss: 2.5773
updat

### 5. Test the Trained Network

In [60]:
saver.restore(sess, file_path + '/model')

risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

for p, p_time in enumerate(pred_time):
    pred_horizon = int(p_time)
    result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

    for t, t_time in enumerate(eval_time):                
        eval_horizon = int(t_time) + pred_horizon
        for k in range(num_Event):
            result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
   
    if p == 0:
        final1, final2 = result1, result2
    else:
        final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)
        
        
row_header = []
for p_time in pred_time:
    for k in range(num_Event):
        row_header.append('pred_time {}: event_{}'.format(p_time,k+1))
            
col_header = []
for t_time in eval_time:
    col_header.append('eval_time {}'.format(t_time))

# c-index result
df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

# brier-score result
df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

### PRINT RESULTS
print('========================================================')
print('--------------------------------------------------------')
print('- C-INDEX: ')
print(df1)
print('--------------------------------------------------------')
print('- BRIER-SCORE: ')
print(df2)
print('========================================================')

INFO:tensorflow:Restoring parameters from PBC2/model
--------------------------------------------------------
- C-INDEX: 
                      eval_time 10
pred_time 8: event_1      0.731543
--------------------------------------------------------
- BRIER-SCORE: 
                      eval_time 10
pred_time 8: event_1      0.055628


### AUC Calculation

In [61]:
def prediction_df(risk_all, event):
    """
    Gets the individual predictions from risk_all
    event = 0 -> 
    """
    
    for i in range(len(risk_all[event])): 
        
        chunk = risk_all[event][i].tolist()
        chunk = pd.DataFrame(chunk)
        chunk['pt_id'] = i
        if i == 0:
            df = chunk
        else: 
            df = pd.concat([df, chunk])

    df = df.reset_index()
    df = df.rename(columns = {'index':'pred_time'})
    df['pred_time'] = df['pred_time'] * 2 + 1 # need to change if eval or pred time is edited

    df = pd.melt(df, id_vars=['pt_id', 'pred_time'], value_vars=[0]).sort_values(['pt_id', 'pred_time', 'variable'])

    df = df.rename(columns = {'variable' : 'eval_time'})
    
    return df

def prediction_auc(pred_t, eval_t , event_n, df):
    """Calculates the AUC at each prediction time and prediction time"""
    
    global true 
    
    time_horizon = pred_t + eval_t 
    true = (te_time <= 15) * (te_label == 1).astype(int)

    auc = roc_auc_score(true.tolist(), df.loc[(df.pred_time == pred_t) & (df.eval_time == eval_t), 'value'].tolist())
            
    return auc 

def prediction_pred_values(pred_t, eval_t , event_n, df):
    """Gets the predication values and true labels for AUC CI calculation"""
    
    global true 
    
    time_horizon = pred_t + eval_t 
    true = (te_time <= time_horizon) * (te_label == event_n).astype(int)

    pred_values = df.loc[(df.pred_time == pred_t) & (df.eval_time == eval_t), 'value'].tolist()
            
    pred_values_df = pd.DataFrame(true, columns = ['true_labels'])
    pred_values_df['pred_values'] = pred_values    
    
    return pred_values_df

def prediction_ROC(pred_t, eval_t , event_n, df):
    """Calculates the AUC at each prediction time and prediction time"""
    
    global true 
    
    time_horizon = pred_t + eval_t 
    true = (te_time <= 15) * (te_label == 1).astype(int)

    fpr, tpr, thresh = roc_curve(true.tolist(), df.loc[(df.pred_time == pred_t) & (df.eval_time == eval_t), 'value'].tolist())
            
    return fpr, tpr, thresh

def get_all_auc(risk_all):
    """Get the auc matrix"""

    pred_df = prediction_df(risk_all, 0)

    for p, p_t in enumerate(pred_time):
        p_t_row = []
        for e, e_t in enumerate(eval_time):
            p_t_row.append(round(prediction_auc(p_t, e, 1, pred_df),3))
        if p == 0:
            auc_df = pd.DataFrame(p_t_row).T
        else: 
            auc_df = pd.concat([auc_df, pd.DataFrame(p_t_row).T], axis = 0)

    auc_df.columns = ['eval_time_' + str(i) for i in eval_time]
    auc_df.index = ['pred_time_' + str(i) for i in pred_time]
    
    return auc_df, pred_df

In [62]:
# Get individual risks for AUC calculation
pred_df = prediction_df(risk_all, 0)
pred_values_5_10 = prediction_auc(1,0,0,pred_df)
pred_values_5_10
#os.chdir('C:/Users/excen/Dropbox/HSIP/Research/MESA/Results/AUC Values')
#pred_values_5_10.to_csv('pred_values_5_10.csv', index = False)

0.7283588629392761

## PCE Calculation

In [ ]:
import ASCVD_Calc_PCE
from importlib import reload
reload(ASCVD_Calc_PCE)

In [ ]:
# get risk measurements from PCE data set (Multiple)
pred_time_index = 2
pce_pred_df, pce_df = ASCVD_Calc_PCE.pce_pred_df_tab(te_data_raw, pred_time, pred_time_index)

# mean risk from PCE equation
np.round(pce_pred_df.risk.mean(),3)

In [ ]:
# generate data for PCE 
pd.DataFrame(te_data_raw[:,3]).to_csv('R:/PrevMed/Projects/MESA_RiskPred/Output/test5.csv')
pd.DataFrame(tr_data_raw[:,3]).to_csv('R:/PrevMed/Projects/MESA_RiskPred/Output/train5.csv')

In [ ]:
# calculate AUC 
def prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster):
    """
    Calculates the AUC of the PCE predictions at a given time
    pred_time: pred time list of prediction times
    pred_time_index: which prediction time in terms of position in the list 
    pce_pred_df: df of probability of PCE dataset
    """
    global true, time_horizon
    
    time_horizon = np.zeros((len(te_label),1))
    time_horizon = [pred_time[pred_time_index] + 10 + i for i in time_horizon]
    true = (te_time <= time_horizon) * (te_label == 1).astype(int)
    
    pce_pred_df['true_label'] = true
    pce_pred_df = pce_pred_df.loc[~pce_pred_df.risk.isnull(),:]
    
    if cluster != 'none':
        pce_pred_df = pce_pred_df.loc[pce_pred_df['labels'] == cluster, :]
        
    return pce_pred_df

def prediction_auc_PCE(pred_time, pred_time_index, pce_pred_df, cluster):
    
    pce_pred_df = prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster)
    auc = roc_auc_score(pce_pred_df['true_label'].tolist(), pce_pred_df['risk'].tolist())
    
    return auc 

def prediction_ROC(pred_time, pred_time_index, pce_pred_df, cluster):
    
    pce_pred_df = prediction_auc_PCE_df(pred_time, pred_time_index, pce_pred_df, cluster)
    fpr, tpr, thresh = roc_curve(pce_pred_df['true_label'].tolist(), pce_pred_df['risk'].tolist())
    
    return fpr, tpr, thresh

In [ ]:
# Get PCE AUC for Each Prediction Year
print('NULL Risk Values from PCE: ', pce_pred_df.risk.isnull().sum())
print('PCE AUC @ Yr 1: ', np.round(prediction_auc_PCE(pred_time, 0, pce_pred_df, 'none'),3))
print('PCE AUC @ Yr 3: ', np.round(prediction_auc_PCE(pred_time, 1, pce_pred_df, 'none'),3))
print('PCE AUC @ Yr 5: ', np.round(prediction_auc_PCE(pred_time, 2, pce_pred_df, 'none'),3))
print('PCE AUC @ Yr 7: ', np.round(prediction_auc_PCE(pred_time, 3, pce_pred_df, 'none'),3))

In [ ]:
# 5 year ROC 
pce_fpr, pce_tpr, pce_thresh = prediction_ROC(pred_time, 2, pce_pred_df, 'none')
auc1 = np.round(prediction_auc_PCE(pred_time, 2, pce_pred_df, 'none'),3)
auc2 = 0.764#np.round(prediction_auc(5, 3, 1, pred_df), 3)

In [ ]:
dh_roc_df = pd.read_csv('/Users/Excenity/Documents/HSIP/Research/Longitudinal PCE/GitHub/PCE_Dynamic-DeepHit/dh_roc_df.csv')
auc3 = 0.703

#### Plot ROC Curve

In [ ]:
plt.plot(pce_fpr, pce_tpr,label='PCE AUC='+str(auc1))
plt.plot(dh_roc_df['fpr'], dh_roc_df['tpr'],label='DH AUC='+str(auc3))
plt.plot(ddh_fpr, ddh_tpr,label='DDH AUC='+str(auc2))
plt.legend(loc=4)

In [ ]:
roc_df = pd.DataFrame({'fpr': ddh_fpr,
                       'tpr': ddh_tpr})
roc_df.to_csv('/Users/Excenity/Documents/HSIP/Research/Longitudinal PCE/GitHub/PCE_Dynamic-DeepHit/ddh_roc_df.csv', index = False)

In [ ]:
dh_fpr, dh_tpr

## Compare Population Groups

1. Get predicted probabilities of risk
2. Get the appropriate labels
3. Get relevant demographics of participants 
4. Create AUCs for respective participants

In [ ]:
### Dynamic DeepHit
# Time Of Interest
pred_time_i = 2
pred_time_ia = 5
eval_time_i = 3
eval_time_ia= 10

# get prediction risk
pred_df = prediction_df(risk_all, 0)
# get only the evaluation time of 10 years
pred_df = pred_df[(pred_df['eval_time'] == eval_time_i) & (pred_df['pred_time'] == pred_time_ia)]

# get true labels
time_horizon = pred_time_ia + eval_time_ia
true = (te_time <= time_horizon) * (te_label == 1).astype(int)
pred_df['true_label'] = true

# get actual demographics data
demo_df = pd.DataFrame(te_data_raw[:,pred_time_i])
demo_df.columns = ['x', 'sbp', 'dbp', 'hdl', 'chol', 'age', 'cig', 'dm03', 'htnmed', 'race', 'gender']

demo_df = demo_df.reset_index()
pred_df = pred_df.reset_index()
final = pd.concat([demo_df, pred_df], axis = 1)

#final = final.drop(columns = ['level_0', 'index', 'x'])

In [ ]:
### PCEs
pce_pred_df = pce_pred_df.reset_index()
pce_pred_df = pce_pred_df[['risk']]
pce_df = pce_df.reset_index()
pce_df['true_label'] = true
final_pce = pd.concat([pce_pred_df, pce_df], axis = 1)

### Race Comparisons
1. Black
2. White and Other Races

In [ ]:
# convert race categories for PCE data
final.loc[final['race'] == 1, 'race'] = 'white'
final.loc[final['race'] == 2, 'race'] = 'asian'
final.loc[final['race'] == 3, 'race'] = 'black'
final.loc[final['race'] == 4, 'race'] = 'hispanic'

In [ ]:
# black comparison
### Dynamic DeepHit
final_t = final.loc[final['race'] == 'black', :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['race'] == 'black', :]
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_black = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_black = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_black.to_csv('pred_values_ddh_black.csv', index = False)
pred_values_pce_black.to_csv('pred_values_pce_black.csv', index = False)

In [ ]:
print(final_t.value.mean())
print(final_pce_t.risk.mean())

In [ ]:
# other comparison
final_t = final.loc[final['race'] != 'black', :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['race'] != 'black', :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_other = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_other = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_other.to_csv('pred_values_ddh_other.csv', index = False)
pred_values_pce_other.to_csv('pred_values_pce_other.csv', index = False)

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ geom_smooth(method='lm', color = 'green')

In [ ]:
print(final_t.value.mean())
print(final_pce_t.risk.mean())

In [ ]:
race_auc = pd.DataFrame({'race':['black', 'black', 'other', 'other'],
                         'method': ['DDH', 'PCE', 'DDH', 'PCE'],
                         'auc': [0.747, 0.734, 0.752, 0.736]})

ggplot(race_auc) + aes(x = 'race', y = 'auc', fill = 'method') + geom_col(position = 'dodge') + ylim (0,1) + geom_text(aes(label='auc'), va = 'bottom', position=position_dodge(width=0.9)) + theme_bw()

### Gender Comparisons
1. Female
2. Male

In [ ]:
# female comparison
### Dynamic DeepHit
final_t = final.loc[final['gender'] == 0, :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['gender'] == 'F', :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_female = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_female = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_female.to_csv('pred_values_ddh_female.csv', index = False)
pred_values_pce_female.to_csv('pred_values_pce_female.csv', index = False)

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

In [ ]:
# male comparison
final_t = final.loc[final['gender'] != 0, :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['gender'] == 'M', :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_male = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_male = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_male.to_csv('pred_values_ddh_male.csv', index = False)
pred_values_pce_male.to_csv('pred_values_pce_male.csv', index = False)

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

In [ ]:
gender_auc = pd.DataFrame({'gender':['F', 'F', 'M', 'M'],
                         'method': ['DDH', 'PCE', 'DDH', 'PCE'],
                         'auc': [0.763, 0.764, 0.735, 0.693]})

ggplot(gender_auc) + aes(x = 'gender', y = 'auc', fill = 'method') + geom_col(position = 'dodge') + ylim (0,1) + geom_text(aes(label='auc'), va = 'bottom', position=position_dodge(width=0.9)) + theme_bw()

### Age Comparisons
1. 45-65
2. 65-85

In [ ]:
# Create Age Groups
final['age_group'] = '44-64'
final.loc[final['age'] > 65, 'age_group'] = '65-85'
final_pce['age_group'] = '44-64'
final_pce.loc[final_pce['age'] > 65, 'age_group'] = '65-85'

In [ ]:
# young comparison
### Dynamic DeepHit
final_t = final.loc[final['age_group'] == '44-64', :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['age_group'] == '44-64', :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_young = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_young = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_young.to_csv('pred_values_ddh_young.csv', index = False)
pred_values_pce_young.to_csv('pred_values_pce_young.csv', index = False)

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

In [ ]:
# old comparison
final_t = final.loc[final['age_group'] == '65-85', :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['age_group'] == '65-85', :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
pred_values_ddh_old = final_t.loc[:, ['true_label', 'value']]
pred_values_pce_old = final_pce_t.loc[:, ['true_label', 'risk']]

pred_values_ddh_old.to_csv('pred_values_ddh_old.csv', index = False)
pred_values_pce_old.to_csv('pred_values_pce_old.csv', index = False)

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

In [ ]:
age_auc = pd.DataFrame({'age':['44-64', '44-64', '65-85', '65-85'],
                         'method': ['DDH', 'PCE', 'DDH', 'PCE'],
                         'auc': [0.752, 0.624, 0.710, 0.654]})

ggplot(age_auc) + aes(x = 'age', y = 'auc', fill = 'method') + geom_col(position = 'dodge') + ylim (0,1) + geom_text(aes(label='auc'), va = 'bottom', position=position_dodge(width=0.9)) + theme_bw()

### Risk Group Comparisons
1. Above 7.5% Group

In [ ]:
# Label Risk Group
final_pce['primary_risk'] = 0
final_pce.loc[final_pce['risk']>=0.075, 'primary_risk'] = 1
final['primary_risk'] = 0
final.loc[final_pce['primary_risk'] == 1, 'primary_risk'] = 1

In [ ]:
# high comparison
### Dynamic DeepHit
final_t = final.loc[final['primary_risk'] == 1, :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['primary_risk'] == 1, :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

In [ ]:
# low comparison
final_t = final.loc[final['primary_risk'] == 0, :]
print('DDH AUC: ', round(roc_auc_score(final_t['true_label'].tolist(), final_t['value'].tolist()),3))
### PCE
final_pce_t = final_pce.loc[final_pce['primary_risk'] == 0, :]
final_pce_t = final_pce_t.dropna()
print('PCE AUC: ', round(roc_auc_score(final_pce_t['true_label'].tolist(), final_pce_t['risk'].tolist()),3))

In [ ]:
final_tt = final_t.iloc[:,[0,16]]
final_pce_tt = final_pce_t.loc[: ,['index', 'risk']]
comp_df = pd.merge(final_tt, final_pce_tt, how = 'left', on = 'index')

ggplot(comp_df) + aes(x = 'risk', y = 'value') + geom_point(alpha = .25, color = 'green') + theme_bw() + xlab('PCE Risk') + ylab('DDH Risk') #+ xlim(0,1) + ylim(0,1) #+ geom_smooth(method='lm') 

### Visualization

In [ ]:
from visualization import get_data, viz_dataset, dup_list, create_outcome_df, create_pce_df, create_outcome_final_df, viz_plot

In [ ]:
patient_id = 273
death_pred , ascvd_pred, pce_data = get_data(patient_id, risk_all, te_data_raw)
pce_data = create_pce_df(pce_data)
outcome_df = create_outcome_final_df(ascvd_pred, death_pred, pred_time, eval_time)

In [ ]:
viz_plot(outcome_df, pce_data, te_label[patient_id][0], te_time[patient_id][0], pred_time)